**Ejercicio 1**. Crea una clase que represente un bloque dentro de nuestra estructura de blockchain. Esta clase se llamará "Block" deberá tener como atributos:

*   index: que representa el índice dentro de la cadena de bloques.
*   timestamp: marca de tiempo sobre el bloque.
*   data: información que va a contener el bloque.
*   previous_hash: hash del bloque anterior.
*   nonce: numero entero que nos va a permitir el minado.
*   hash: hash del bloque.

**Ejercicio 2.** Crea un constructor (`__init__`) que inicialice dichos atributos a partir de: index, timestamp, data y previous_hash.

**Ejercicio 3.** Dentro de la clase Block, debes crear las siguientes funciones:

* ```def calculate_hash(self)``` Devolverá el hash del bloque. Puedes hacer uso de la función `hashlib.sha256(<str>)` a la que se le pasa un string por parámetro.

* ```def __str__(self)``` Devolverá un string con información de todos los atributos del bloque: index, timestamp, data, previous_hash, nonce, y hash.

* ```def mine_block(self, difficulty)``` Función que servirá para minar el bloque. El minado consistirá en ir cambiando el valor de nonce hasta que el hash obtenido tenga tantos ceros al principio como indique el parámetro `difficulty`.  





In [1]:
import hashlib
import time

In [2]:
class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.nonce = 0
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_content = f"{self.index}{self.timestamp}{self.data}{self.previous_hash}{self.nonce}".encode()
        return hashlib.sha256(block_content).hexdigest()

    def mine_block(self, difficulty):
        target = '0' * difficulty
        while not self.hash.startswith(target):
            self.nonce += 1
            self.hash = self.calculate_hash()

    def __str__(self):
        return f"Block(Index: {self.index}, Timestamp: {self.timestamp}, Data: {self.data}, Previous Hash: {self.previous_hash}, Nonce: {self.nonce}, Hash: {self.hash})"


**Ejercicio 4**. Crea una clase llamada "Blockchain" que tenga como atributos:

*   chain: es una lista de objetos de tipo Block, donde el primer bloque seá el bloque génesis.
*   difficulty: es la dificultad (número entero) que tendrá la cadena de bloques para ir minando nuevos bloques.


**Ejercicio 5**. Crea un constructor de Blockchain que le asigne por defecto una dificultad 4 y el bloque génesis. Este bloque tendrá como índice el 0, como timestamp el momento de la llamada, data será "génesis block" y el previous_hash será 0.


**Ejercicio 6**. Crea dentro de la clase Blockchain los siguientes métodos:

*  `def create_genesis_block(self)` devolverá un bloque con índice igual a 0, como timestamp el momento de la llamada, data será "génesis block" y el previous_hash será 0.

* `def get_latest_block(self)` devolverá el último bloque de la cadena.

* `def is_chain_valid(self)` devolverá verdadero si la cadena de blockchain es correcta o no.

* `def add_block(self, new_block)` minará new_block para poder ser introducido a la cadena de blockchain.


In [3]:
class Blockchain:
    def __init__(self):
        self.difficulty = 4
        self.chain = [self.create_genesis_block()]

    def create_genesis_block(self):
        return Block(0, time.time(), "Genesis Block", "0")

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, new_block):
        new_block.previous_hash = self.get_latest_block().hash
        new_block.mine_block(self.difficulty)
        self.chain.append(new_block)

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]
            if current_block.hash != current_block.calculate_hash():
                return False
            if current_block.previous_hash != previous_block.hash:
                return False
        return True


**Ejercicio 7**. Crea una cadena de blockchain y genera 3 bloques válidos.

In [10]:

# Ejemplo de uso
blockchain = Blockchain()
blockchain.add_block(Block(1, time.time(), "Block 1 Data", blockchain.get_latest_block().hash))
blockchain.add_block(Block(2, time.time(), "Block 2 Data", blockchain.get_latest_block().hash))
blockchain.add_block(Block(3, time.time(), "Block 3 Data", blockchain.get_latest_block().hash))

for block in blockchain.chain:
    print(block)

Block(Index: 0, Timestamp: 1743084571.7298968, Data: Genesis Block, Previous Hash: 0, Nonce: 0, Hash: 471f6aeb8d5c22c06b174b6c27eb35bdc69ae25c395d18bd5f44c9e7c1508470)
Block(Index: 1, Timestamp: 1743084571.7299614, Data: Block 1 Data, Previous Hash: 471f6aeb8d5c22c06b174b6c27eb35bdc69ae25c395d18bd5f44c9e7c1508470, Nonce: 180647, Hash: 00001ebac4005428125ffb8a168240fb81a5811e981b2f714ff3398759ff0a08)
Block(Index: 2, Timestamp: 1743084572.0922358, Data: Block 2 Data, Previous Hash: 00001ebac4005428125ffb8a168240fb81a5811e981b2f714ff3398759ff0a08, Nonce: 39602, Hash: 0000772a21df1c6d78fc81e6a7c870a765e0e14baf5a39f7d4ffd4d77559d1e1)
Block(Index: 3, Timestamp: 1743084572.170819, Data: Block 3 Data, Previous Hash: 0000772a21df1c6d78fc81e6a7c870a765e0e14baf5a39f7d4ffd4d77559d1e1, Nonce: 21423, Hash: 00000d3d6ad5aa5e2e183b8eb55cbabdfabfbd035bc74dc68983b9f0e2589a89)


# Mejoras para convertir el sistema en distribuido.

El sistema se podría convertir en distribuido implementando un mecanismo en el que diferentes nodos colaboren para validar y propagar los bloques. Posibles formas:

1. Nodos de la red: Cada nodo ejecutaría una copia del código de la cadena de bloques y sería capaz de recibir y transmitir bloques a otros nodos en la red.

2. Protocolo de consenso: Necesitarías implementar un protocolo de consenso para que los nodos acuerden qué cadena de bloques es la válida cuando hay múltiples versiones. Algunas opciones comunes incluyen Prueba de Trabajo (Proof of Work), Prueba de Participación (Proof of Stake), o algoritmos de consenso más avanzados como Practical Byzantine Fault Tolerance (PBFT) o Delegated Proof of Stake (DPoS).

3. Transmisión de bloques: Necesitarías establecer un mecanismo para que los nodos puedan transmitir bloques a través de la red. Esto podría hacerse a través de una red peer-to-peer (P2P) utilizando sockets o protocolos específicos como HTTP o WebSockets.

4. Los nodos deben verificar que los bloques que reciben son válidos antes de agregarlos a su propia cadena. Esto implica verificar la estructura del bloque, su hash, su validez en términos de reglas de consenso, y su relación con la cadena existente.

5. **Incentivos: Podrías considerar la introducción de un sistema de incentivos para motivar a los nodos a participar honestamente en el mantenimiento de la red. Esto podría incluir recompensas en forma de criptomonedas o tarifas de transacción.**